# Learning Latvian!!
-----------------------------------------

In [223]:
import urllib.request, sys, time
from bs4 import BeautifulSoup
import requests
from newspaper import Article, nlp
import re, string
import pandas as pd
from googletrans import Translator
#pip install googletrans==4.0.0-rc1
from nltk import tokenize
translator = Translator()

Enter the URL below (code currently configured to work with LSM.lv)

In [199]:
url ='https://www.lsm.lv/'

Response should be 200

In [217]:
new_df= pd.read_csv('latvian_translation.csv')
trs_builder = dict(zip(new_df.LV, new_df.EN))

In [200]:
try:
    page = requests.get(url)
    print('Page Response: ', page)
    
except Exception as e:
    error_type, error_obj, error_info = sys.exc_info()
    print('ERROR FOR LINK:',url)
    print(error_type, error_obj, error_info.tb_lineno)

Page Response:  <Response [200]>


_Get URLS with class "class_tag"- (configured to lsm)_ Also removes '#'

In [201]:
soup = BeautifulSoup(page.text, 'lxml')

class_tags = 'thumbnail__caption'
holder = []

for tags in soup.find_all('a',class_tags):
    holder.append(tags['href'])
    
holder.remove('#')

## Converts from relative to absolute URLS
## Truncate URL... remove everything from ?utm* -- Specific to LSM. Will need to be tweaked/skipped for other websites
cleaned_url = []

for words in holder[:15]:
    if words[:1] == '/':
        new_page = 'https://www.lsm.lv'+ words
        cleaned_url.append(new_page.split('?utm')[0])

For each URL, get link, title, summary. Then split summary into individual words, remove any punctuations and parse via Google Translate

_To Do_  
  - Break down words to its root and translate the root word only

In [238]:
kw_builder = []

for cleaned in cleaned_url[:2]:
    print(cleaned)
    article = Article(cleaned)
    article.download()
    article.parse()
    article.nlp()
    print(article.title,'\n')
    print(article.meta_description,'\n')
    a_words = article.meta_description.split()
    cleaned_words = [''.join(letter for letter in word if letter not in string.punctuation) for word in a_words if word]
    
    sentences = tokenize.sent_tokenize(article.meta_description)
    
    for word in cleaned_words:
        if len(word) > 3:
            translated_text = translator.translate(word, src='lv', dest='en')
            print(word, ' ->', translated_text.text)
            
            
            trs_builder.update({word : translated_text.text})
            kw_builder.append(word)
    
    print('--'*50)        
    for sentence in sentences:
        trans_sentence = translator.translate(sentence, src='lv', dest='en' )
        print(sentence)
        print( trans_sentence.text)
    print('--'*50)    
    print('*'*50)

https://www.lsm.lv/raksts/zinas/latvija/jaunas-vienotibas-premjera-amata-kandidats-nakamajas-saeimas-velesanas-bus-karins.a462063/
«Jaunās vienotības» premjera amata kandidāts nākamajās Saeimas vēlēšanās būs Kariņš 

Rudenī gaidāmajās 14. Saeimas vēlēšanās partiju apvienības "Jaunā vienotība" Ministru prezidenta amata kandidāts būs esošais premjers Krišjānis Kariņš. 

Rudenī  -> In the fall
gaidāmajās  -> in the upcoming
Saeimas  -> Saeima
vēlēšanās  -> desire
partiju  -> party
apvienības  -> association
Jaunā  -> New
vienotība  -> unity
Ministru  -> Minister
prezidenta  -> presidential
amata  -> position
kandidāts  -> candidate
esošais  -> Existing
premjers  -> prime minister
Krišjānis  -> Krišjānis
Kariņš  -> War
----------------------------------------------------------------------------------------------------
Rudenī gaidāmajās 14.
In the upcoming 14th.
Saeimas vēlēšanās partiju apvienības "Jaunā vienotība" Ministru prezidenta amata kandidāts būs esošais premjers Krišjānis Kariņš.


In [219]:
# Remove duplicates
temp = []
res = dict()
for key, val in trs_builder.items():
    if val not in temp:
        temp.append(val)
        res[key] = val

In [221]:
df = pd.DataFrame(res.items(), columns=['LV','EN'])
df.to_csv('latvian_translation.csv', index=False)